# StyleGrid I

Extending the [DataGrid](./DataGrid.ipynb), `StyleGrid` adds support for top-level styles of the rendered grid. Advanced customization is possible with [custom renderers](./StyleGrid%20II.ipynb).

In [ ]:
import pandas as pd, traitlets as T, ipywidgets as W
import importnb
from wxyz.html import AlphaColorPicker 
from wxyz.datagrid import StyleGrid

In [ ]:
with importnb.Notebook():
    try:
        from DataGrid import make_grid
    except:
        from .DataGrid import make_grid

In [ ]:
def make_style_grid_i(GridClass=StyleGrid, df=None, container=W.VBox):
    pg = make_grid(GridClass, df=df)
    dg = pg.dg

    rs = W.IntSlider(description='row height', orientation='vertical')
    cs = W.IntSlider(description='column width')
    hr = W.HTML()
    hc = W.HTML()

    W.jslink((dg, "row_size"), (rs, "value"))
    W.jslink((dg, "column_size"), (cs, "value"))
    colors = [ 
        [p, W.jslink((dg, t), (p, "value"))][0]
        for t in dg.traits().keys()
        for p in [AlphaColorPicker(description=t.replace('_color', '').replace('_', ' '))]
        if t.endswith('_color')
    ]
    grid_playground = W.HBox([
        W.VBox([
            W.HBox([W.HBox([hr, hc]), cs]), 
            pg, 
        ], layout=dict(flex="1")),
        W.VBox([rs, *colors]), 
    ])
    grid_playground.dg = dg
    return grid_playground

In [ ]:
if __name__ == "__main__":
    display(make_style_grid_i())